In [1]:
from rag_shap2 import*
import pandas as pd
from scipy.stats import pearsonr
import time

/home/cs.aau.dk/em63by/anaconda3/envs/llmx/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
documents = [
    "The weather in Paris is sunny today.",
    "Paris is the capital of France.",
    "The sun is shining in Paris today",
    "Berlin is the capital of Germany.", # Irrelevant
    # "The Eiffel Tower is located in Paris, France.",
    # "France borders several countries including Germany.",
    # "The currency used in France is the Euro.",
    # "Paris hosted the Summer Olympics in 1900 and 1924.",
    # "Germany uses the Euro as well.", # Redundant info
    "It is cloudy in Berlin today." # Irrelevant
]
query = "What is the weather like in the capital of France?"
target_response = "The weather in Paris is sunny." # The ideal answer fragment

attributor = ShapleyAttributor(
        items=documents,
        query=query,
        target_response=target_response,
        llm_caller=compute_logprob # Use your actual llm_caller
    )

print("\n--- Computing ContextCite (Surrogate Weights) ---")
cc_weights = attributor.compute(method_name="contextcite", num_samples=16, lasso_alpha=0.01)
if cc_weights is not None: print("ContextCite Weights:", np.round(cc_weights, 4))

print("\n--- Computing Weakly Supervised Shapley (WSS) ---")
wss_values, wss_surrogate_weights = attributor.compute(method_name="wss", num_samples=16, lasso_alpha=0.01, return_weights=True)
if wss_values is not None:
    print("WSS Values:", np.round(wss_values, 4))
    print("WSS Surrogate Weights:", np.round(wss_surrogate_weights, 4))

print("\n--- Computing TMC-Shapley ---")
tmc_values = attributor.compute(method_name="tmc") # 5*n iterations
if tmc_values is not None: print("TMC Values:", np.round(tmc_values, 4))

if beta_dist: # Only if scipy is available
    print("\n--- Computing Beta-Shapley (U-shaped, more weight to ends) ---")
    # For U-shaped, alpha and beta < 1
    beta_u_values = attributor.compute(method_name="betashap", num_iterations=attributor.n_items, beta_a=0.5, beta_b=0.5)
    if beta_u_values is not None: print("BetaShap (U-shaped) Values:", np.round(beta_u_values, 4))

    print("\n--- Computing Beta-Shapley (Uniform, equivalent to standard MC) ---")
    beta_uniform_values = attributor.compute(method_name="betashap", num_iterations=attributor.n_items, beta_a=1.0, beta_b=1.0)
    if beta_uniform_values is not None: print("BetaShap (Uniform) Values:", np.round(beta_uniform_values, 4))
print("\n--- Computing Leave-One-Out (LOO) ---")
loo_values = attributor.compute(method_name="loo") 
if loo_values is not None: 
    print("LOO Values:", np.round(loo_values, 4))

print("\n--- Computing Exact Shapley ---")
exact_values = attributor.compute(method_name="exact", exact_confirm=False) # Disable confirm for n=5 demo
if exact_values is not None: print("Exact Values:", np.round(exact_values, 4))

if exact_values is not None:
    if wss_values is not None:
        mae_wss = np.mean(np.abs(exact_values - wss_values))
        print(f"\nMAE WSS vs Exact: {mae_wss:.4f}")
    if tmc_values is not None:
        mae_tmc = np.mean(np.abs(exact_values - tmc_values))
        print(f"MAE TMC vs Exact: {mae_tmc:.4f}")
    if beta_dist and beta_u_values is not None:
        mae_beta_u = np.mean(np.abs(exact_values - beta_u_values))
        print(f"MAE BetaShap (U) vs Exact: {mae_beta_u:.4f}")
    if beta_dist and beta_uniform_values is not None:
        mae_beta_uni = np.mean(np.abs(exact_values - beta_uniform_values))
        print(f"MAE BetaShap (Uni) vs Exact: {mae_beta_uni:.4f}")
    # Compare LOO to Exact if available
    if exact_values is not None and loo_values is not None:
        mae_loo = np.mean(np.abs(exact_values - loo_values))
        print(f"\nMAE LOO vs Exact: {mae_loo:.4f}")



--- Computing ContextCite (Surrogate Weights) ---
Starting contextcite (n=5, m=16)...
Computing true utilities for 16 samples...


contextcite LLM Calls: 100%|██████████| 16/16 [02:40<00:00, 10.03s/it]


Training surrogate model...
  Surrogate Weights (w): [ 5.1677 -0.164   4.8074  1.0443  4.0094]
  Surrogate Intercept (b): -23.2409
Method 'contextcite' finished in 160.51s.
ContextCite Weights: [ 5.1677 -0.164   4.8074  1.0443  4.0094]

--- Computing Weakly Supervised Shapley (WSS) ---
Starting wss (n=5, m=16)...
Computing true utilities for 16 samples...


wss LLM Calls: 100%|██████████| 16/16 [02:27<00:00,  9.20s/it]


Training surrogate model...
  Surrogate Weights (w): [ 4.7911  0.9767  3.9839 -0.9505  5.64  ]
  Surrogate Intercept (b): -22.8280
Building hybrid utility set for WSS...
Calculating Shapley values from hybrid utilities for WSS...


...

BetaShap Iterations: 100%|██████████| 5/5 [02:25<00:00, 29.03s/it]


Beta-Shapley made 17 unique LLM calls.
Method 'betashap' finished in 153.27s.
BetaShap (Uniform) Values: [11.917   0.742   2.2346  1.0273  2.6947]

--- Computing Leave-One-Out (LOO) ---
Starting Leave-One-Out (LOO) computation (n=5)...
  Calculating utility of full set V(N)...
  Calculating utility V(N-i) for 5 items...


LOO computation made 6 unique LLM calls.
Method 'loo' finished in 57.06s.
LOO Values: [ 1.5858 -1.3239  0.1629 -0.2838  0.3653]

--- Computing Exact Shapley ---
Starting Exact Shapley (n=5, 32 LLM calls)...


Exact LLM Calls: 100%|██████████| 32/32 [04:51<00:00,  9.11s/it]
                                                                    

Method 'exact' finished in 291.42s.
Exact Values: [7.0016 1.5099 4.6902 1.0479 4.3661]

MAE WSS vs Exact: 1.1251
MAE TMC vs Exact: 3.2490
MAE BetaShap (U) vs Exact: 4.7033
MAE BetaShap (Uni) vs Exact: 1.9662

MAE LOO vs Exact: 3.6218


In [ ]:
from scipy.stats import spearmanr
correlation, p_value = spearmanr(shapley_values, lasso_weights)
print(f"Spearman Correlation: {correlation:.4f}, p-value: {p_value:.4f}\n")

In [ ]:
logp, p, model_out, token_probs = compute_logprob(
    query="What is the capital of Kazakhstan?",
    ground_truth_answer="Paris",
    return_token_probs=True
)

print("Log probability:", logp)
print("Total probability:", p)
print("Model generated:", model_out)